In [59]:
import clinicalPicture as cp
import matplotlib.pyplot as plt
from itertools import product
from disease import DiseaseExemple, SignOfDiseaseExemple
from typing import TypedDict

In [60]:
PD_COUNT_MAX = 6

In [61]:
def createArayDelimiters(original_array_length: int, count_dilimetrs: int):
    if count_dilimetrs == 0:
        return [[]]
    stop = [*range(original_array_length-count_dilimetrs, original_array_length)]
    arr  = []
    
    def create(word, i_let):
        while word[i_let] < stop[i_let]+1:
            if len(word) < count_dilimetrs:
                create([*word, word[-1]+1], i_let+1)
            else:
                arr.append(word.copy())
            word[i_let] += 1
    
    create([1], 0)
    
    return arr

In [62]:
def checkIntersections(a0: tuple[float, float], a1: tuple[float, float]) -> bool:
    a0 = sorted(a0)
    a1 = sorted(a1)
    ans = not (a0[1] < a1[0] or a1[1] < a0[0]) 
    return ans

In [63]:
class PosiblePD(TypedDict):
    zdp:         list
    pd_duration: list[tuple[int]]


class AllPsiblePD(dict[str, list[PosiblePD]]):
    pass


class SignType(str):
    def __init__(self, s) -> None:
        if not (s in ['discrete', 'continuous']):
            raise TypeError('SignType')
        super().__init__(s)

In [64]:
def findePosiblePD(exemple: SignOfDiseaseExemple, 
                   sign_type: SignType) -> AllPsiblePD:
    arr_val         = exemple['value']
    arr_time        = exemple['time']
    periods_dinamic = {f'{pd_count+1}':[] for pd_count in range(PD_COUNT_MAX)}
    
    for count_dilimetrs_pd in range(PD_COUNT_MAX):
    
        delimiters_arr = createArayDelimiters(len(arr_val), count_dilimetrs_pd)

        for delimiters in delimiters_arr:
            
            delimiters.append(None)
            delimiter_prev = 0
            v = []
            
            for delimiter in delimiters:
                
                if sign_type == 'discrete':
                    v.append(set(arr_val[delimiter_prev:delimiter]))
                    delimiter_prev = delimiter
                    if len(v)<2:
                        continue
                    union_pd = v[-1] & v[-2]
                    if len(union_pd) != 0:
                        break
                
                elif sign_type == 'continuous':
                    sample = (arr_val[delimiter_prev:delimiter])
                    v.append((min(sample), max(sample)))
                    delimiter_prev = delimiter
                    if len(v)<2:
                        continue
                    
                    if checkIntersections(v[-2], v[-1]):
                        break
                    
                else:
                    raise TypeError(f'{sign_type} not SignType')

            else:
                delimiter_prev = 0
                time = []
                for delimiter in delimiters:
                    times = arr_time[delimiter_prev:delimiter]
                    delimiter_prev = delimiter
                    time.append((times[0], times[-1]))
                    
                periods_dinamic[f'{count_dilimetrs_pd+1}'].append({'zdp': v, 'pd_duration': time})
    return periods_dinamic

In [65]:
def unionPD(all_posibles_pd_1: AllPsiblePD,
            all_posibles_pd_2: AllPsiblePD,
            sign_type: SignType):

    def unionVal(arr1: list, arr2: list, sign_type: SignType):
        ans = []
        if sign_type == 'discrete':
            for a, b in zip(arr1, arr2):
                ans.append(a | b)
                if len(ans)<2:
                    continue
                
                union_pd = ans[-1] & ans[-2]
                if len(union_pd) != 0:
                    return None
        elif sign_type == 'continuous':
            for a, b in zip(arr1, arr2):
                temp = [*a, *b]
                ans.append((min(temp), max(temp)))
                if len(ans)<2:
                    continue
                if checkIntersections(ans[-2], ans[-1]):
                    return None
        else:
            raise TypeError(f'{sign_type} not SignType')
            
        return ans
    
    
    def unionTime(arr1, arr2):
        ans = []
        for a, b in zip(arr1, arr2):
            all_val = [*a, *b]
            ans.append((min(all_val), max(all_val)))
        return ans
    
    
    ans = {}
    for count_pd in range(1, PD_COUNT_MAX+1):
        ans[f'{count_pd}'] = []
        for combinations in product(all_posibles_pd_1[f'{count_pd}'], all_posibles_pd_2[f'{count_pd}']):
            union_val = unionVal(combinations[0]['zdp'], combinations[1]['zdp'], sign_type)
            if union_val is not None:
                union_time = unionTime(combinations[0]['pd_duration'], combinations[1]['pd_duration'])
                ans[f'{count_pd}'].append({'zdp': union_val, 'pd_duration': union_time})
    return ans

In [247]:
def interpret_pd_time(arr: list[tuple[int, int]]):
    # [(1, 23), (3, 44), (20, 57), (24, 77)]
    ans = []
    for i in range(len(arr)):
        if i != 0:
            max = arr[i][1] - arr[i-1][1]
        else:
            max = arr[i][1]
            
        if i != len(arr)-1:
            min = arr[i+1][0] - arr[i][0]
        else:
            min = int(max / 2)
            
            
        ans.append((min, max))
    return ans

In [ ]:
def print_posible_pd(all_posible_pd: AllPsiblePD):
    for desease_name in all_posible_pd:
        print(desease_name)
        for sign_name in all_posible_pd[desease_name]:
            print('\t', sign_name)
            for count_pd in all_posible_pd[desease_name][sign_name]:
                print('\t\t', count_pd, ' периодов:')
                for pd in all_posible_pd[desease_name][sign_name][count_pd]:
                    
                    print('\t\t\t', pd['zdp'], '  //  ' , interpret_pd_time(pd['pd_duration']))

In [140]:
g = cp.ClinicalPicture(signs_count=2, disease_count=1)
print(type(g))
g.generateSings(boundaries_value_exponent_cont = [2, 3],
                               part_discr_cont = 0.5,
                         count_value_max_discr = 20)
g.generateDisease(pd_count_max=PD_COUNT_MAX)

<class 'clinicalPicture.ClinicalPicture'>


In [141]:
a, b, c = g.data_frame
c

НГ  ВГ                            ЗДП
заболевание признак       номер ПД                                       
болезнь 0   дискретный 1  1          3  12    [1, 2, 4, 6, 7, 11, 12, 14]
                          2          2  17          [3, 8, 9, 10, 13, 15]
                          3         12  18                  [1, 2, 5, 14]
                          4         14  23  [0, 3, 6, 10, 11, 12, 13, 15]
            непрерывный 1 1         18  21                   [-4.5, 39.5]
                          2         12  19                  [-17.6, -7.5]
                          3         21  22                    [8.3, 32.1]
                          4         11  16                   [-21.2, 6.4]
                          5          1  14                 [-37.0, -21.4]

In [142]:
first_example = g.createExample(count_mesurment_in_pd_max=3)

In [159]:
posible_pd = {}
for d in g.disease:
    posible_pd[f'{d.name}'] = {}
    for s in g.signs:
        posible_pd[f'{d.name}'][f'{s.name}'] = {}

for disease_name in first_example:
    disease_exemple = first_example[disease_name]['signs_discrete']
    for sign_name in disease_exemple:
        posible_pd[disease_name][sign_name] = findePosiblePD(disease_exemple[sign_name], 'discrete')

    disease_exemple = first_example[disease_name]['signs_continuous']
    for sign_name in disease_exemple:
        posible_pd[disease_name][sign_name] = findePosiblePD(disease_exemple[sign_name], 'continuous')
print_posible_pd(posible_pd)

болезнь 0
	 дискретный 1
		 1  периодов:
			 [{1, 3, 8, 9, 13, 14}]   //   [(45, 47)]
		 2  периодов:
			 [{1, 3, 8, 9, 14}, {13}]   //   [(45, 29), (0, 45)]
		 3  периодов:
			 [{1}, {9}, {1, 3, 8, 13, 14}]   //   [(13, 2), (1, 13), (31, 32)]
			 [{1}, {9, 3}, {8, 1, 13, 14}]   //   [(13, 2), (2, 14), (30, 32)]
			 [{1}, {8, 9, 3}, {1, 13, 14}]   //   [(13, 2), (8, 15), (24, 32)]
			 [{1}, {8, 9, 3, 14}, {1, 13}]   //   [(13, 2), (13, 21), (19, 32)]
			 [{1, 9}, {3}, {8, 1, 13, 14}]   //   [(14, 15), (1, 14), (30, 31)]
			 [{1, 9}, {8, 3}, {1, 13, 14}]   //   [(14, 15), (7, 15), (24, 31)]
			 [{1, 9}, {8, 3, 14}, {1, 13}]   //   [(14, 15), (12, 21), (19, 31)]
			 [{1, 3, 9}, {8}, {1, 13, 14}]   //   [(15, 16), (6, 15), (24, 30)]
			 [{1, 3, 9}, {8, 14}, {1, 13}]   //   [(15, 16), (11, 21), (19, 30)]
			 [{8, 1, 3, 9}, {14}, {1, 13}]   //   [(21, 17), (5, 21), (19, 24)]
		 4  периодов:
			 [{1}, {9}, {3}, {8, 1, 13, 14}]   //   [(13, 2), (1, 13), (1, 1), (30, 31)]
			 [{1}, {9}, {8, 3}

In [354]:
for _ in range(10):
    example = g.createExample(count_mesurment_in_pd_max=3)
    for disease_name in example:
        
        disease_exemple = example[disease_name]['signs_discrete']
        for sign_name in disease_exemple:
            posible = findePosiblePD(disease_exemple[sign_name], 'discrete')
            posible_pd[disease_name][sign_name] = unionPD(posible_pd[disease_name][sign_name], posible, 'discrete')
            
        disease_exemple = example[disease_name]['signs_continuous']
        for sign_name in disease_exemple:
            posible = findePosiblePD(disease_exemple[sign_name], 'continuous')
            posible_pd[disease_name][sign_name] = unionPD(posible_pd[disease_name][sign_name], posible, 'continuous')
            
print_posible_pd(posible_pd)
            

болезнь 0
	 дискретный 1
		 1  периодов:
			 [{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}]   //   [(34, 68)]
		 2  периодов:
		 3  периодов:
		 4  периодов:
			 [{1, 2, 4, 6, 7, 11, 12, 14}, {3, 8, 9, 10, 13, 15}, {1, 2, 5, 14}, {0, 3, 6, 10, 11, 12, 13, 15}]   //   [(3, 12), (3, 17), (12, 17), (11, 22)]
		 5  периодов:
		 6  периодов:
	 непрерывный 1
		 1  периодов:
			 [(-37.0, 39.5)]   //   [(45, 91)]
		 2  периодов:
			 [(-21.2, 39.5), (-37.0, -21.4)]   //   [(62, 78), (6, 13)]
		 3  периодов:
		 4  периодов:
			 [(-4.4, 39.5), (-17.6, -7.5), (8.3, 32.1), (-37.0, 6.4)]   //   [(18, 21), (12, 19), (21, 22), (14, 29)]
		 5  периодов:
			 [(-4.4, 39.5), (-17.6, -7.5), (8.3, 32.1), (-21.2, 6.4), (-37.0, -21.4)]   //   [(18, 21), (12, 19), (21, 22), (11, 16), (6, 13)]
		 6  периодов:
